#### Basic package & directory settings
- network_wrangler branch: [bicounty_ml_connectors](https://github.com/wsp-sag/network_wrangler/tree/bicounty_ml_connectors)
- Lasso branch: [bicounty_emme](https://github.com/wsp-sag/Lasso/tree/bicounty_emme)

In [ ]:
import os
import sys
import yaml
import pickle
import numpy as np
import pandas as pd
import geopandas as gpd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger
from network_wrangler.transitnetwork import DotDict

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc
from lasso import util
from lasso import bicounty

import logging
logger = logging.getLogger("WranglerLogger")
logger.setLevel(logging.INFO)

%load_ext autoreload
%autoreload 2

In [ ]:
# TODO: user input
scenario_year = 2050

data_dir = "../data/"
lasso_dir = "../software/Lasso/"
parameters = Parameters(lasso_base_dir = lasso_dir)

standard_net_dir = os.path.join(data_dir, 'scenario_files', 'standard_networks', 'version_00')
interim_data_dir = os.path.join(data_dir, 'scenario_files', 'interim')
project_card_dir = os.path.join(data_dir, 'project_cards')

#### Load raw scenario pickle file
- this is the scenario pickle file where no project card is applied yet

In [ ]:
pickle_file_name = os.path.join(data_dir, "scenario_files", "scenario_pickle", "raw_scenario.pickle")
raw_scenario = pickle.load(open(pickle_file_name, 'rb'))

In [ ]:
with open(f"{data_dir}/project_card_list.yml", "r") as file:
    project_card_info = yaml.safe_load(file)

project_cards_lookup = {}
project_cards_dir_lookup = {}

for i in project_card_info:
    card_set_name = i["card_set"]
    project_cards_lookup[card_set_name] = i["projects"]
    project_cards_dir_lookup[card_set_name] = i["card_dir"]

In [ ]:
# helper function for reading all project cards in the scenario project list
def get_project_card_list(
    card_set: str,
    project_card_dir: str=project_card_dir, 
    project_cards_lookup: dict=project_cards_lookup,
    project_cards_dir_lookup: dict=project_cards_dir_lookup
):
    # return a list of project card objects
    result_project_card_list = []
    project_card_name_list = project_cards_lookup[card_set]
    for card_name in project_card_name_list:
        _filename = os.path.join(project_card_dir, project_cards_dir_lookup[card_set], card_name)
        card = ProjectCard.read(_filename, validate=False)
        result_project_card_list.append(card)
    return result_project_card_list

### Apply project cards for different scenario years

#### Scenario 2015
- create `scenario_2015`
- apply project card set `2015_highway`
- apply project card set `2015_transit`
- add centroid/external connectors
- apply project card set `2015_externals`
- add pnr connectors
- apply project card set `2015_others`
- if scenario_year is 2015
    - apply project card set `for_2015_only`

In [ ]:
scenario_2015 = Scenario.create_scenario(base_scenario = raw_scenario)

##### Apply scenario 2015 highway project cards
- These are mtc correction project cards with following tags:
  - highway review
  - Major Arterial Review
  - Reversible Lanes
  - Bus Only
  - Toll Plaza
  - Managed Lanes
  - toll review
  - Exclude Trucks
  - Major Transit links

In [ ]:
card_2015_highway_list = get_project_card_list(card_set="2015_highway")

for item in card_2015_highway_list:
    scenario_2015.apply_project(item)

##### Apply scenario 2015 transit project cards
- These are mtc correction project cards with following tags:
  - Minor Transit
  - Add Transit
  - Major Transit
  - Toll Plaza Transit

In [ ]:
# set roadnet for transit_net before applying transit project cards
scenario_2015.transit_net.set_roadnet(scenario_2015.road_net, validate_consistency = False)

card_2015_transit_list = get_project_card_list(card_set="2015_transit")

for item in card_2015_transit_list:
    scenario_2015.apply_project(item)

##### Add centroid/external connectors from standard_networks & apply related project cards

In [ ]:
scenario_2015.road_net = mtc.add_centroid_and_centroid_connector(
    roadway_network = scenario_2015.road_net,
    parameters = parameters,
    centroid_file=os.path.join(standard_net_dir, 'centroid_node.pickle'),
    centroid_connector_link_file=os.path.join(standard_net_dir, 'cc_link.pickle'),
    centroid_connector_shape_file=os.path.join(standard_net_dir, 'cc_shape.pickle')
)

##### Apply scenario 2015 externals project cards
- These are mtc correction project cards with following tag:
  - External Stations Review

In [ ]:
card_2015_externals_list = get_project_card_list(card_set="2015_externals")

for item in card_2015_externals_list:
    scenario_2015.apply_project(item)

In [ ]:
# add county
scenario_2015.road_net = mtc.calculate_county(
    roadway_network = scenario_2015.road_net,
    parameters = parameters,
    network_variable = 'county'
)

# remove MTC externals in SJ county
remove_external_node_list = [900011, 900012, 900013]
scenario_2015.road_net.nodes_df = scenario_2015.road_net.nodes_df[
    ~(scenario_2015.road_net.nodes_df.model_node_id.isin(remove_external_node_list))
]
scenario_2015.road_net.links_df = scenario_2015.road_net.links_df[
    ~(
        (scenario_2015.road_net.links_df.A.isin(remove_external_node_list)) |
        (scenario_2015.road_net.links_df.B.isin(remove_external_node_list))
    )
]
scenario_2015.road_net.shapes_df = scenario_2015.road_net.shapes_df[
    scenario_2015.road_net.shapes_df.id.isin(
        scenario_2015.road_net.links_df.id.tolist())
]

##### Build PNR connectors

In [ ]:
road_net_with_pnr = bicounty.build_pnr_connections(scenario_2015.road_net, parameters, build_pnr_taz_connector=False)

In [ ]:
# update model node id for pnr
node_id_outside_of_cube_cap = road_net_with_pnr.nodes_df[road_net_with_pnr.nodes_df.model_node_id >= 10000000].model_node_id.unique().tolist()

node_id_outside_of_cube_cap_dict = dict(
    zip(node_id_outside_of_cube_cap, range(5300000, 5300000 + len(node_id_outside_of_cube_cap) + 1))
)

# replace model node id
road_net_with_pnr.nodes_df['model_node_id'] = np.where(
    road_net_with_pnr.nodes_df['model_node_id'].isin(node_id_outside_of_cube_cap),
    road_net_with_pnr.nodes_df['model_node_id'].map(node_id_outside_of_cube_cap_dict),
    road_net_with_pnr.nodes_df['model_node_id']
)

road_net_with_pnr.links_df['A'] = np.where(
    road_net_with_pnr.links_df['A'].isin(node_id_outside_of_cube_cap),
    road_net_with_pnr.links_df['A'].map(node_id_outside_of_cube_cap_dict),
    road_net_with_pnr.links_df['A']
)

road_net_with_pnr.links_df['B'] = np.where(
    road_net_with_pnr.links_df['B'].isin(node_id_outside_of_cube_cap),
    road_net_with_pnr.links_df['B'].map(node_id_outside_of_cube_cap_dict),
    road_net_with_pnr.links_df['B']
)

In [ ]:
# set extra nodes as pnr
road_net_with_pnr.nodes_df.loc[road_net_with_pnr.nodes_df["model_node_id"].isin([2192948, 2625972, 2526973, 2625974, 2625975, 3097285, 3097286]), "pnr"] = 1

In [ ]:
# update 2015 scenario using roadway network with pnr
scenario_2015 = Scenario.create_scenario(base_scenario= {"road_net": road_net_with_pnr, "transit_net": scenario_2015.transit_net})

##### Apply extra 2015 project cards

In [ ]:
card_2015_extra_list = get_project_card_list(card_set="2015_others")

for item in card_2015_extra_list:
    scenario_2015.apply_project(item)

##### If scenario year = 2015, apply remaining project cards only for 2015
- These are managed lane project cards that have different settings from 2020 onward

In [ ]:
if scenario_year == 2015:
    card_2015_only_list = get_project_card_list(card_set="for_2015_only")

    for item in card_2015_only_list:
        scenario_2015.apply_project(item)

#### scenario 2020
- if scenario_year >= 2020
    - create `scenario_2020`
    - apply project card set `2020_highway`
    - apply project card set `2020_transit`

In [ ]:
if scenario_year >= 2020:
    # create scenario 2020
    scenario_2020 = Scenario.create_scenario(base_scenario = scenario_2015)

    # apply 2020 highway cards
    card_2020_highway_list = get_project_card_list(card_set="2020_highway")
    for item in card_2020_highway_list:
        scenario_2020.apply_project(item)

    # apply 2020 transit cards
    scenario_2020.transit_net.set_roadnet(scenario_2020.road_net, validate_consistency = False) # set roadnet for transit
    card_2020_transit_list = get_project_card_list(card_set="2020_transit")
    for item in card_2020_transit_list:
        scenario_2020.apply_project(item)

#### Scenario 2035 & 2050 - part 1: standard format highway project cards
- if scenario_year >= 2035
  - apply project card set `2035_standard_format`
- if scenario_year is 2050
  - apply project card set `2050_standard_format`

In [ ]:
if scenario_year >= 2035:
    # create scenario 2035
    scenario_2035 = Scenario.create_scenario(base_scenario = scenario_2020)

    # apply 2035 highway cards (standard format)
    card_2035_standard_highway_list = get_project_card_list(card_set="2035_standard_format")
    for item in card_2035_standard_highway_list:
        scenario_2035.apply_project(item)

In [ ]:
if scenario_year == 2050:
    # create scenario 2050
    scenario_2050 = Scenario.create_scenario(base_scenario = scenario_2035)

    # apply 2050 highway cards (standard format)
    card_2050_standard_highway_list = get_project_card_list(card_set="2050_standard_format")
    for item in card_2050_standard_highway_list:
        scenario_2050.apply_project(item)

#### Convert to model network format

In [ ]:
if scenario_year == 2015:
    curr_scenario = scenario_2015
elif scenario_year == 2020:
    curr_scenario = scenario_2020
elif scenario_year == 2035:
    curr_scenario = scenario_2035
else: # scenario_year == 2050
    curr_scenario = scenario_2050

In [ ]:
model_roadway_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = curr_scenario.road_net, 
    parameters = parameters
)

In [ ]:
# extra step to shorten the "name" field
model_roadway_net.links_df['name'] = model_roadway_net.links_df['name'].apply(lambda x: "" if type(x) == int else x)
model_roadway_net.links_df['name'] = model_roadway_net.links_df['name'].apply(lambda x: util.shorten_name(x))

In [ ]:
# convert to model network format
# all managed lane parallel links and time period attributes are created in this step
model_net = mtc.roadway_standard_to_mtc_network(model_roadway_net, parameters=parameters)

In [ ]:
# One link in the network has a null geometry. Deleting it and adding it back again with a diferent project card
model_net.links_mtc_df=model_net.links_mtc_df[model_net.links_mtc_df['geometry'].notnull()]

In [ ]:
# extra step to shorten the "name" field
model_net.links_mtc_df['name'] = model_net.links_mtc_df['name'].apply(lambda x: "" if type(x) == int else x)
model_net.links_mtc_df['name'] = model_net.links_mtc_df['name'].apply(lambda x: util.shorten_name(x))

#### Scenario 2035 & 2050 - part 2: model network format highway project cards & transit project cards
- if scenario_year is 2035
    - apply project card set `2035_model_net_format`
    - apply project card set `2035_transit`
- if scenario_year is 2050
    - apply project card set `2035_model_net_format`
    - apply project card set `2050_model_net_format`
    - apply project card set `2035_transit`
    - apply project card set `2050_transit`

In [ ]:
# update curr_scenario so that extra project cards can be applied further
if scenario_year >= 2035:
    # update links_df and nodes_df in curr_scenario
    curr_scenario.road_net.links_df = model_net.links_mtc_df.copy()
    curr_scenario.road_net.nodes_df = model_net.nodes_mtc_df.copy()

    # also set "model_node_id" be "N"
    curr_scenario.road_net.nodes_df['model_node_id'] = curr_scenario.road_net.nodes_df['N']

    # set gdf
    curr_scenario.road_net.links_df = gpd.GeoDataFrame(curr_scenario.road_net.links_df, geometry=curr_scenario.road_net.links_df.geometry)
    curr_scenario.road_net.nodes_df = gpd.GeoDataFrame(curr_scenario.road_net.nodes_df, geometry=curr_scenario.road_net.nodes_df.geometry)

In [ ]:
if scenario_year == 2035:
    # apply 2035 highway cards (model network format)
    card_2035_model_net_highway_list = get_project_card_list(card_set="2035_model_net_format")
    for item in card_2035_model_net_highway_list:
        curr_scenario.apply_project(item)

    # other special treatment
    # TODO: copied from previous notebooks, but should consider removing this part
    curr_scenario.road_net.links_df['model_link_id'] = np.where((curr_scenario.road_net.links_df['A']==1511000)&(curr_scenario.road_net.links_df['B']==6011000),
                                                                curr_scenario.road_net.links_df['model_link_id'].max()+1,
                                                                curr_scenario.road_net.links_df['model_link_id'])

    curr_scenario.road_net.links_df['model_link_id'] = np.where((curr_scenario.road_net.links_df['A']==3031452)&(curr_scenario.road_net.links_df['B']==3078980),
                                                                curr_scenario.road_net.links_df['model_link_id'].max()+1,
                                                                curr_scenario.road_net.links_df['model_link_id'])

    # apply 2035 transit cards
    curr_scenario.transit_net.set_roadnet(curr_scenario.road_net, validate_consistency = False) # set roadnet for transit
    card_2035_transit_list = get_project_card_list(card_set="2035_transit")
    for item in card_2035_transit_list:
        curr_scenario.apply_project(item)

In [ ]:
if scenario_year == 2050:
    # apply 2035 highway cards (model network format)
    card_2035_model_net_highway_list = get_project_card_list(card_set="2035_model_net_format")
    for item in card_2035_model_net_highway_list:
        curr_scenario.apply_project(item)    

    # apply 2050 highway cards (model network format)
    card_2050_model_net_highway_list = get_project_card_list(card_set="2050_model_net_format")
    for item in card_2050_model_net_highway_list:
        curr_scenario.apply_project(item)    

    # other special treatment
    # TODO: copied from previous notebooks, but should consider removing this part
    curr_scenario.road_net.links_df['model_link_id'] = np.where((curr_scenario.road_net.links_df['A']==1511000)&(curr_scenario.road_net.links_df['B']==6011000),
                                                                curr_scenario.road_net.links_df['model_link_id'].max()+1,
                                                                curr_scenario.road_net.links_df['model_link_id'])

    curr_scenario.road_net.links_df['model_link_id'] = np.where((curr_scenario.road_net.links_df['A']==3031452)&(curr_scenario.road_net.links_df['B']==3078980),
                                                                curr_scenario.road_net.links_df['model_link_id'].max()+1,
                                                                curr_scenario.road_net.links_df['model_link_id'])

    # apply 2035 transit cards
    curr_scenario.transit_net.set_roadnet(curr_scenario.road_net, validate_consistency = False) # set roadnet for transit
    card_2035_transit_list = get_project_card_list(card_set="2035_transit")
    for item in card_2035_transit_list:
        curr_scenario.apply_project(item)

    # apply 2050 transit cards
    card_2050_transit_list = get_project_card_list(card_set="2050_transit")
    for item in card_2050_transit_list:
        curr_scenario.apply_project(item)

### Export results

#### Add `BRT` attribute if it's not in roadway links (specifically for 2015)
- the first project card that has `BRT` field is in 2020, but this field is required for all model years

In [ ]:
if "BRT" not in model_net.links_mtc_df.columns:
    print("add BRT placeholder field")
    model_net.links_mtc_df["BRT"] = 0

#### Write out result to scenario pickle file (2015 & 2020)

In [ ]:
# note: for 2035 and 2050, the scenario file is already in model network format, so only write out scenario pickle for 2015 & 2020
if scenario_year <= 2020:
    if scenario_year == 2015:
        result_scenario = scenario_2015
    else:
        result_scenario = scenario_2020

    scenario_filename = os.path.join(data_dir, 'scenario_files', 'scenario_pickle', f'scenario_{scenario_year}.pickle')
    pickle.dump(result_scenario, open(scenario_filename, 'wb'))

#### Write out Cube transit network
- note: writing out cube transit.lin requires [faresystem_crosswalk.txt](https://github.com/wsp-sag/Lasso/blob/bicounty_emme/mtc_data/lookups/faresystem_crosswalk.txt) in the output directory

In [ ]:
# update transit net
curr_scenario.update_transit_net_with_new_road_net()

In [ ]:
# write out cube transit net
standard_transit_net = StandardTransit.fromTransitNetwork(curr_scenario.transit_net, parameters=parameters)
mtc.write_as_cube_lin(standard_transit_net, parameters, outpath = os.path.join(data_dir, 'scenario_files', 'model_networks', f"{scenario_year}", 'trn', "transit.lin"))

#### Read the `transit.lin` written out in the previous step and add the `has_transit` attribute to the highway network

In [ ]:
# collect all transit links from parsing through transit.lin file
transit_file = os.path.join(data_dir, 'scenario_files', 'model_networks', f"{scenario_year}", 'trn', "transit.lin")
# gather network links by parsing through the node sequences in transit.lin
with open(transit_file, "r") as f:
    lines = f.readlines()

    all_line_links = pd.DataFrame()
    curr_line = None
    line_node_seq = None

    for txt in lines:
        if txt.startswith("LINE NAME="):
            # print (txt.split("=")[1])
            # store the current line name
            curr_line = txt.split("\"")[1]
            # reset line_node_seq as an empty list
            line_node_seq = []

        # add to node sequence if the first item of txt after split by "," and remove whitespace is digit
        # if txt.strip().split(",")[0].replace(" ", "").replace("-", "").isdigit():
        if txt.strip().split(",")[0].replace("-", "").isdigit():
            node = int(txt.strip().split(",")[0].replace("-", ""))
            line_node_seq.append(node)

        if curr_line and txt == "\n":
            print(f"processed line: {curr_line}")
            # convert previous line_node_seq into df
            line_links = pd.DataFrame({"line": curr_line, "A": line_node_seq[:-1], "B": line_node_seq[1:]})
            # add to all_line_links
            all_line_links = pd.concat([all_line_links, line_links]).reset_index(drop=True)

all_line_links["A"] = all_line_links["A"].astype(int)
all_line_links["B"] = all_line_links["B"].astype(int)
all_line_links = all_line_links.drop_duplicates(['A','B'])
all_line_links['has_transit'] = 1

In [ ]:
if scenario_year >= 2035:
    # add `has_transit` attribute to scenario roadway network
    has_transit = pd.merge(all_line_links, curr_scenario.road_net.links_df[['A','B','model_link_id']])
    curr_scenario.road_net.links_df['has_transit'] = curr_scenario.road_net.links_df['model_link_id'].map(dict(zip(has_transit['model_link_id'],has_transit['has_transit']))).fillna(0)
else: # scenario_year = 2015 or 2020
    # add `has_transit` attribute to model_net
    has_transit = pd.merge(all_line_links, model_net.links_mtc_df[['A','B','model_link_id']])
    model_net.links_mtc_df['has_transit'] = model_net.links_mtc_df['model_link_id'].map(dict(zip(has_transit['model_link_id'],has_transit['has_transit']))).fillna(0)

#### Write out Cube highway network

In [ ]:
if scenario_year >= 2035:
    # `nodes_mtc_df` and `links_mtc_df` are the gdfs that will be written out to cube network
    model_net_updated = ModelRoadwayNetwork.from_RoadwayNetwork(
        roadway_network_object = curr_scenario.road_net, 
        parameters = parameters
    )
else:
    model_net_updated = model_net

model_net_updated = mtc.calculate_farezone(
    roadway_network = model_net_updated,
    transit_network = curr_scenario.transit_net,
    parameters = parameters,
    network_variable = 'farezone',
    overwrite = True,
)

if scenario_year >= 2035:
    model_net_updated.nodes_mtc_df = model_net_updated.nodes_df.copy()
    model_net_updated.links_mtc_df = model_net_updated.links_df.copy()

In [ ]:
# extra step to shorten the "name" field
model_net_updated.links_mtc_df['name'] = model_net_updated.links_mtc_df['name'].apply(lambda x: "" if type(x) == int else x)
model_net_updated.links_mtc_df['name'] = model_net_updated.links_mtc_df['name'].apply(lambda x: util.shorten_name(x))

model_net_updated.write_roadway_as_fixedwidth(
    output_dir = os.path.join(data_dir, 'scenario_files', 'model_networks', f"{scenario_year}", 'hwy'),
    output_link_txt = 'links.txt',
    output_node_txt = 'nodes.txt',
    output_link_header_width_txt = 'links_header_width.txt',
    output_node_header_width_txt = 'nodes_header_width.txt',
    output_cube_network_script = 'make_complete_network_from_fixed_width_file.s',
    output_cube_network_name = 'complete_network_with_externals.net'
    #drive_only = True
)

#### Write out link true-shape shapefile

In [ ]:
model_net_updated.links_mtc_df[[
    'A', 'B', 'model_link_id', 'distance',
    'name', 'roadway', 'county', 'cntype',
    'ft', 'assignable',
    'drive_access', 'walk_access', 
    'rail_only', 'bus_only', 'transit', 'nmt2010','nmt2020',
    'lanes_EA', 'lanes_AM', 'lanes_MD', 'lanes_PM', 'lanes_EV',
    'managed', 'tollbooth', 'tollseg', 
    'useclass_AM', 'useclass_EA', 'useclass_EV', 'useclass_MD', 'useclass_PM', 
    'BRT', 'has_transit',
    'geometry'
]].to_file(os.path.join(data_dir, 'scenario_files', 'model_networks', f"{scenario_year}", 'hwy', f"complete_network_{scenario_year}.shp"))